In [ ]:
# 금리 예측 상담 서비스 - Google Colab 버전
# 필요한 패키지 설치
!pip install yfinance prophet langchain openai pyngrok langchain-community

In [ ]:
pip install streamlit

In [ ]:
#  기본 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import yfinance as yf
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
import warnings
warnings.filterwarnings('ignore')

from google.colab import files
import io

# Streamlit 대신 Colab에서 사용 가능한 인터페이스 설정
from pyngrok import ngrok
import streamlit as st

# Ngrok 인증 (무료 계정인 경우 필요)
!ngrok authtoken Y1ou9hCwZym4yBH78OO3gSuUjuMC_6hG8KvaSsRuCPVjsmjDAC

class InterestRatePredictionService:
    def __init__(self, api_key=None):
        """
        금리 예측 상담 서비스 초기화

        Parameters:
        api_key (str): OpenAI API 키
        """
        self.data = None
        self.model_prophet = None
        self.model_lstm = None
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.api_key = api_key
        self.initialize_llm()

    def initialize_llm(self):
        """LLM 및 대화 체인 초기화"""
        if self.api_key:
            template = """
            당신은 금융 전문가로서 금리에 관한 상담을 제공합니다.

            현재 상황:
            - 현재 기준금리: {current_rate}%
            - 3개월 후 예상 금리: {prediction_3m}% (신뢰구간: {confidence_3m}%)
            - 6개월 후 예상 금리: {prediction_6m}% (신뢰구간: {confidence_6m}%)
            - 1년 후 예상 금리: {prediction_1y}% (신뢰구간: {confidence_1y}%)

            최근 금리 동향: {trend_description}

            이전 대화 내용:
            {history}

            사용자: {input}
            금융 전문가:
            """

            prompt = PromptTemplate(
                input_variables=["history", "input", "current_rate", "prediction_3m",
                                "prediction_6m", "prediction_1y", "confidence_3m",
                                "confidence_6m", "confidence_1y", "trend_description"],
                template=template
            )

            self.llm = OpenAI(temperature=0.7, openai_api_key=self.api_key)
            self.memory = ConversationBufferMemory()
            self.conversation = ConversationChain(
                llm=self.llm,
                memory=self.memory,
                prompt=prompt
            )
        else:
            self.llm = None
            print("API 키가 제공되지 않았습니다. LLM 기능은 비활성화됩니다.")

    def get_interest_rate_data(self, source='fred', ticker='DGS10', start_date='2000-01-01', uploaded_file=None):
        """
        금리 데이터 수집

        Parameters:
        source (str): 데이터 소스 ('fred', 'yahoo', 또는 'upload')
        ticker (str): 데이터 티커
        start_date (str): 시작 날짜
        uploaded_file: 업로드된 CSV 파일 (선택 사항)

        Returns:
        pd.DataFrame: 금리 데이터
        """
        try:
            # 파일 업로드 옵션 처리
            if uploaded_file is not None:
                # CSV 파일 읽기
                content = uploaded_file.read().decode('utf-8')
                data = pd.read_csv(io.StringIO(content), parse_dates=True, index_col=0)
                data = data.rename(columns={data.columns[0]: 'Rate'})
            else:
                # 온라인 데이터 소스에서 가져오기
                data = yf.download(f"{ticker}=X" if source == 'yahoo' else ticker,
                                  start=start_date,
                                  end=datetime.now().strftime('%Y-%m-%d'))

                if source == 'yahoo':
                    data = data[['Close']].rename(columns={'Close': 'Rate'})
                else:
                    data = data[['Adj Close']].rename(columns={'Adj Close': 'Rate'})

            # 결측치 처리
            data = data.fillna(method='ffill')
            self.data = data
            print(f"{len(data)} 개의 데이터 포인트를 가져왔습니다.")
            return data

        except Exception as e:
            print(f"데이터 수집 중 오류 발생: {e}")
            return None

    # 나머지 메서드는 원래 코드와 동일

    # 이하 preprocess_data, train_prophet_model, create_lstm_sequences, train_lstm_model,
    # predict_future_rates, visualize_forecast, get_consultation 메서드 동일

    def preprocess_data(self):
        """
        데이터 전처리 수행

        Returns:
        tuple: 전처리된 데이터 세트
        """
        if self.data is None:
            print("먼저 get_interest_rate_data 메서드를 호출하여 데이터를 가져오세요.")
            return None

        # 시계열 인덱스 확인
        self.data.index = pd.to_datetime(self.data.index)

        # Prophet 모델 준비
        prophet_data = self.data.reset_index().rename(columns={'Date': 'ds', 'Rate': 'y'})

        # LSTM 모델 준비
        dataset = self.data.values
        scaled_data = self.scaler.fit_transform(dataset)

        return prophet_data, scaled_data

    def train_prophet_model(self, periods=365):
        """
        Prophet 모델을 학습하고 예측 수행

        Parameters:
        periods (int): 예측할 미래 기간(일)

        Returns:
        pd.DataFrame: 예측 결과
        """
        prophet_data, _ = self.preprocess_data()

        # Prophet 모델 학습
        self.model_prophet = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            changepoint_prior_scale=0.05,
            seasonality_prior_scale=10.0
        )
        self.model_prophet.fit(prophet_data)

        # 미래 예측 수행
        future = self.model_prophet.make_future_dataframe(periods=periods)
        forecast = self.model_prophet.predict(future)

        print("Prophet 모델 학습 완료")
        return forecast

    def create_lstm_sequences(self, data, time_steps=60):
        """
        LSTM 모델을 위한 시퀀스 생성

        Parameters:
        data (numpy.ndarray): 스케일링된 데이터
        time_steps (int): 시퀀스 길이

        Returns:
        tuple: X, y 데이터셋
        """
        X, y = [], []
        for i in range(len(data) - time_steps):
            X.append(data[i:(i + time_steps), 0])
            y.append(data[i + time_steps, 0])
        return np.array(X), np.array(y)

    def train_lstm_model(self, time_steps=60, epochs=50, batch_size=32, validation_split=0.2):
        """
        LSTM 모델 학습

        Parameters:
        time_steps (int): 시퀀스 길이
        epochs (int): 학습 에포크 수
        batch_size (int): 배치 크기
        validation_split (float): 검증 데이터 비율

        Returns:
        tf.keras.models.Sequential: 학습된 LSTM 모델
        """
        _, scaled_data = self.preprocess_data()

        # 시퀀스 데이터 생성
        X, y = self.create_lstm_sequences(scaled_data, time_steps)

        # 훈련/테스트 분할
        train_size = int(len(X) * 0.8)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]

        # 데이터 형태 변환 (LSTM 입력용)
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

        # LSTM 모델 구성
        self.model_lstm = Sequential()
        self.model_lstm.add(LSTM(units=50, return_sequences=True, input_shape=(time_steps, 1)))
        self.model_lstm.add(Dropout(0.2))
        self.model_lstm.add(LSTM(units=50, return_sequences=False))
        self.model_lstm.add(Dropout(0.2))
        self.model_lstm.add(Dense(units=1))

        self.model_lstm.compile(optimizer='adam', loss='mean_squared_error')

        # 모델 학습
        history = self.model_lstm.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=validation_split,
            verbose=1
        )

        # 모델 평가
        y_pred = self.model_lstm.predict(X_test)
        y_pred_original = self.scaler.inverse_transform(y_pred)
        y_test_original = self.scaler.inverse_transform(y_test.reshape(-1, 1))

        mse = mean_squared_error(y_test_original, y_pred_original)
        mae = mean_absolute_error(y_test_original, y_pred_original)

        print(f"LSTM 모델 학습 완료")
        print(f"테스트 세트 MSE: {mse:.4f}, MAE: {mae:.4f}")

        return self.model_lstm, history

    def predict_future_rates(self, days=365):
        """
        Prophet 및 LSTM 모델을 사용한 금리 예측

        Parameters:
        days (int): 예측할 미래 기간(일)

        Returns:
        dict: 예측 결과
        """
        if self.model_prophet is None:
            print("먼저 Prophet 모델을 학습하세요.")
            return None

        # Prophet 예측 결과
        forecast = self.model_prophet.predict(self.model_prophet.make_future_dataframe(periods=days))

        # 현재 금리, 3개월, 6개월, 1년 후 예측
        current_date = datetime.now().strftime('%Y-%m-%d')
        current_rate = forecast[forecast['ds'] == current_date]['yhat'].values[0] if current_date in forecast['ds'].dt.strftime('%Y-%m-%d').values else forecast.iloc[-len(forecast) + len(self.data)]['yhat']

        prediction_3m = forecast[forecast['ds'] == (datetime.now() + timedelta(days=90)).strftime('%Y-%m-%d')]['yhat'].values[0]
        prediction_6m = forecast[forecast['ds'] == (datetime.now() + timedelta(days=180)).strftime('%Y-%m-%d')]['yhat'].values[0]
        prediction_1y = forecast[forecast['ds'] == (datetime.now() + timedelta(days=365)).strftime('%Y-%m-%d')]['yhat'].values[0]

        # 신뢰 구간 계산 (Prophet)
        confidence_3m = (forecast[forecast['ds'] == (datetime.now() + timedelta(days=90)).strftime('%Y-%m-%d')]['yhat_upper'].values[0] -
                         forecast[forecast['ds'] == (datetime.now() + timedelta(days=90)).strftime('%Y-%m-%d')]['yhat_lower'].values[0]) / 2
        confidence_6m = (forecast[forecast['ds'] == (datetime.now() + timedelta(days=180)).strftime('%Y-%m-%d')]['yhat_upper'].values[0] -
                         forecast[forecast['ds'] == (datetime.now() + timedelta(days=180)).strftime('%Y-%m-%d')]['yhat_lower'].values[0]) / 2
        confidence_1y = (forecast[forecast['ds'] == (datetime.now() + timedelta(days=365)).strftime('%Y-%m-%d')]['yhat_upper'].values[0] -
                         forecast[forecast['ds'] == (datetime.now() + timedelta(days=365)).strftime('%Y-%m-%d')]['yhat_lower'].values[0]) / 2

        # 금리 동향 설명
        last_3m_data = self.data.iloc[-90:] if len(self.data) >= 90 else self.data
        trend = "상승" if last_3m_data['Rate'].iloc[-1] > last_3m_data['Rate'].iloc[0] else "하락"
        avg_change = (last_3m_data['Rate'].iloc[-1] - last_3m_data['Rate'].iloc[0]) / len(last_3m_data) * 100

        trend_description = f"최근 3개월 동안 금리는 {trend} 추세를 보이고 있으며, 평균 일일 변화율은 {avg_change:.4f}%입니다."

        predictions = {
            'current_rate': round(current_rate, 2),
            'prediction_3m': round(prediction_3m, 2),
            'prediction_6m': round(prediction_6m, 2),
            'prediction_1y': round(prediction_1y, 2),
            'confidence_3m': round(confidence_3m, 2),
            'confidence_6m': round(confidence_6m, 2),
            'confidence_1y': round(confidence_1y, 2),
            'trend_description': trend_description,
            'forecast': forecast
        }

        return predictions

    def visualize_forecast(self, forecast, days_to_show=365, save_path=None):
        """
        예측 결과 시각화

        Parameters:
        forecast (pd.DataFrame): Prophet 예측 결과
        days_to_show (int): 표시할 기간(일)
        save_path (str): 이미지 저장 경로

        Returns:
        matplotlib.figure.Figure: 그래프 피겨
        """
        plt.figure(figsize=(14, 8))

        # 실제 데이터 플롯
        plt.plot(self.data.index[-days_to_show:],
                self.data['Rate'].values[-days_to_show:],
                label='실제 금리', color='blue')

        # 예측 데이터 플롯 (현재부터)
        future_dates = pd.date_range(start=datetime.now(), periods=365)
        future_idx = [i for i, date in enumerate(forecast['ds']) if date >= datetime.now()]

        plt.plot(forecast['ds'].iloc[future_idx],
                forecast['yhat'].iloc[future_idx],
                label='예측 금리', color='red', linestyle='--')

        # 신뢰 구간 표시
        plt.fill_between(forecast['ds'].iloc[future_idx],
                         forecast['yhat_lower'].iloc[future_idx],
                         forecast['yhat_upper'].iloc[future_idx],
                         color='red', alpha=0.2, label='95% 신뢰 구간')

        plt.title('금리 예측 그래프', fontsize=16)
        plt.xlabel('날짜', fontsize=12)
        plt.ylabel('금리 (%)', fontsize=12)
        plt.legend()
        plt.grid(True, alpha=0.3)

        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')

        return plt.gcf()

    def get_consultation(self, user_query, predictions=None):
        """
        사용자 쿼리에 기반한 상담 제공

        Parameters:
        user_query (str): 사용자 질문
        predictions (dict): 예측 결과

        Returns:
        str: LLM 기반 응답
        """
        if self.llm is None:
            return "API 키가 설정되지 않아 LLM 상담 기능을 사용할 수 없습니다."

        if predictions is None:
            predictions = self.predict_future_rates()

        if predictions is None:
            return "먼저 모델을 학습하고 예측을 수행해야 합니다."

        try:
            response = self.conversation.predict(
                input=user_query,
                current_rate=predictions['current_rate'],
                prediction_3m=predictions['prediction_3m'],
                prediction_6m=predictions['prediction_6m'],
                prediction_1y=predictions['prediction_1y'],
                confidence_3m=predictions['confidence_3m'],
                confidence_6m=predictions['confidence_6m'],
                confidence_1y=predictions['confidence_1y'],
                trend_description=predictions['trend_description']
            )
            return response
        except Exception as e:
            return f"상담 제공 중 오류가 발생했습니다: {e}"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Colab에서 Streamlit 실행하기 위한 함수
def run_streamlit_on_colab():
    # 새 app.py 파일 생성
    with open('app.py', 'w') as f:
        f.write('''
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import io
from interest_rate_service import InterestRatePredictionService

st.set_page_config(page_title="금리 예측 상담 서비스", layout="wide")

st.title("금리 예측 상담 서비스")
st.markdown("### 과거 데이터 기반 금리 예측 및 AI 상담 시스템")

with st.sidebar:
    st.header("설정")
    api_key = st.text_input("OpenAI API 키 (선택사항)", type="password")
    data_source = st.selectbox("데이터 소스", ["fred", "yahoo", "upload"])

    if data_source == "upload":
        uploaded_file = st.file_uploader("CSV 파일 업로드", type="csv")
        ticker = None
    else:
        uploaded_file = None
        ticker = st.text_input("티커", value="DGS10")

    start_date = st.date_input("시작 날짜", value=pd.to_datetime("2010-01-01"))

    st.header("모델 설정")
    st.subheader("Prophet 모델")
    forecast_days = st.slider("예측 기간(일)", 30, 730, 365)

    st.subheader("LSTM 모델")
    time_steps = st.slider("시퀀스 길이", 30, 120, 60)
    epochs = st.slider("에포크 수", 10, 100, 50)

    train_button = st.button("모델 학습 시작")

# 메인 컨텐츠
col1, col2 = st.columns([2, 1])

with col1:
    st.header("금리 데이터 및 예측")
    data_tab, forecast_tab = st.tabs(["데이터 분석", "예측 결과"])

    # 서비스 초기화
    if 'service' not in st.session_state:
        st.session_state.service = InterestRatePredictionService(api_key)
        st.session_state.data_loaded = False
        st.session_state.models_trained = False
        st.session_state.predictions = None

    # 데이터 로드
    if not st.session_state.data_loaded or train_button:
        with st.spinner("데이터 로드 중..."):
            data = st.session_state.service.get_interest_rate_data(
                source=data_source,
                ticker=ticker,
                start_date=start_date.strftime('%Y-%m-%d'),
                uploaded_file=uploaded_file
            )
            if data is not None:
                st.session_state.data_loaded = True

    # 데이터 탭 컨텐츠
    with data_tab:
        if st.session_state.data_loaded:
            st.subheader("금리 데이터 시각화")
            fig, ax = plt.subplots(figsize=(10, 6))
            ax.plot(st.session_state.service.data.index, st.session_state.service.data['Rate'])
            ax.set_title("금리 데이터")
            ax.set_xlabel("날짜")
            ax.set_ylabel("금리 (%)")
            ax.grid(True, alpha=0.3)
            st.pyplot(fig)

            st.subheader("통계 요약")
            st.dataframe(st.session_state.service.data.describe())

            # 이동평균 시각화
            st.subheader("이동평균 추세")
            ma_days = st.slider("이동평균 기간(일)", 7, 200, 30)

            fig2, ax2 = plt.subplots(figsize=(10, 6))
            ax2.plot(st.session_state.service.data.index, st.session_state.service.data['Rate'], label='원본 데이터')
            ax2.plot(st.session_state.service.data.index, st.session_state.service.data['Rate'].rolling(window=ma_days).mean(),
                    label=f'{ma_days}일 이동평균', color='red')
            ax2.set_title("금리 이동평균")
            ax2.set_xlabel("날짜")
            ax2.set_ylabel("금리 (%)")
            ax2.legend()
            ax2.grid(True, alpha=0.3)
            st.pyplot(fig2)

    # 모델 학습 및 예측
    if train_button and st.session_state.data_loaded:
        with st.spinner("모델 학습 중..."):
            # Prophet 모델 학습
            st.session_state.service.train_prophet_model(periods=forecast_days)

            # LSTM 모델 학습 (선택적)
            st.session_state.service.train_lstm_model(
                time_steps=time_steps,
                epochs=epochs
            )

            # 예측 수행
            st.session_state.predictions = st.session_state.service.predict_future_rates(days=forecast_days)
            st.session_state.models_trained = True

            st.success("모델 학습 및 예측 완료!")

    # 예측 결과 탭
    with forecast_tab:
        if st.session_state.models_trained and st.session_state.predictions:
            st.subheader("금리 예측 결과")

            col_curr, col_3m, col_6m, col_1y = st.columns(4)

            with col_curr:
                st.metric("현재 금리", f"{st.session_state.predictions['current_rate']}%")

            with col_3m:
                st.metric("3개월 후 예상 금리",
                         f"{st.session_state.predictions['prediction_3m']}% ± {st.session_state.predictions['confidence_3m']}",
                         delta=round(st.session_state.predictions['prediction_3m'] - st.session_state.predictions['current_rate'], 2))

            with col_6m:
                st.metric("6개월 후 예상 금리",
                         f"{st.session_state.predictions['prediction_6m']}% ± {st.session_state.predictions['confidence_6m']}",
                         delta=round(st.session_state.predictions['prediction_6m'] - st.session_state.predictions['current_rate'], 2))

            with col_1y:
                st.metric("1년 후 예상 금리",
                         f"{st.session_state.predictions['prediction_1y']}% ± {st.session_state.predictions['confidence_1y']}",
                         delta=round(st.session_state.predictions['prediction_1y'] - st.session_state.predictions['current_rate'], 2))

            st.subheader("예측 그래프")
            fig = st.session_state.service.visualize_forecast(st.session_state.predictions['forecast'])
            st.pyplot(fig)

            st.markdown(f"**금리 동향 분석**: {st.session_state.predictions['trend_description']}")

# 상담 인터페이스
with col2:
    st.header("AI 금융 상담사")

    if "messages" not in st.session_state:
        st.session_state.messages = [{"role": "assistant", "content": "안녕하세요! 금리에 관한 질문이 있으시면 언제든지 물어보세요."}]

    # 이전 메시지 표시
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # 사용자 입력 처리
    if prompt := st.chat_input("금리에 관해 질문하세요"):
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        with st.chat_message("assistant"):
            if not st.session_state.models_trained:
                response = "아직 모델이 학습되지 않았습니다. 먼저 사이드바에서 '모델 학습 시작' 버튼을 클릭해주세요."
            else:
                with st.spinner("답변 생성 중..."):
                    response = st.session_state.service.get_consultation(prompt, st.session_state.predictions)

            st.markdown(response)

        st.session_state.messages.append({"role": "assistant", "content": response})
        ''')

    # 서비스 클래스를 별도 파일로 저장
    with open('interest_rate_service.py', 'w') as f:
        f.write('''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import yfinance as yf
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
import warnings
import io
warnings.filterwarnings('ignore')

class InterestRatePredictionService:
    # 여기에 위의 서비스 클래스 전체 코드를 붙여넣기
    # (InterestRatePredictionService 클래스 내용)
''')

    # StreamlitTunnel 시작 - ngrok을 통해 외부 접속 허용
    !streamlit run app.py &> /dev/null &
    public_url = ngrok.connect(port=8501)
    print(f"Streamlit 애플리케이션이 다음 URL에서 실행 중입니다: {public_url}")

In [ ]:
# Colab에서 Streamlit 실행하기 위한 함수
def run_streamlit_on_colab():
    # 새 app.py 파일 생성
    with open('app.py', 'w') as f:
        f.write('''
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import io
from interest_rate_service import InterestRatePredictionService

st.set_page_config(page_title="금리 예측 상담 서비스", layout="wide")

st.title("금리 예측 상담 서비스")
st.markdown("### 과거 데이터 기반 금리 예측 및 AI 상담 시스템")

with st.sidebar:
    st.header("설정")
    api_key = st.text_input("OpenAI API 키 (선택사항)", type="password")
    data_source = st.selectbox("데이터 소스", ["fred", "yahoo", "upload"])

    if data_source == "upload":
        uploaded_file = st.file_uploader("CSV 파일 업로드", type="csv")
        ticker = None
    else:
        uploaded_file = None
        ticker = st.text_input("티커", value="DGS10")

    start_date = st.date_input("시작 날짜", value=pd.to_datetime("2010-01-01"))

    st.header("모델 설정")
    st.subheader("Prophet 모델")
    forecast_days = st.slider("예측 기간(일)", 30, 730, 365)

    st.subheader("LSTM 모델")
    time_steps = st.slider("시퀀스 길이", 30, 120, 60)
    epochs = st.slider("에포크 수", 10, 100, 50)

    train_button = st.button("모델 학습 시작")

# 메인 컨텐츠
col1, col2 = st.columns([2, 1])

with col1:
    st.header("금리 데이터 및 예측")
    data_tab, forecast_tab = st.tabs(["데이터 분석", "예측 결과"])

    # 서비스 초기화
    if 'service' not in st.session_state:
        st.session_state.service = InterestRatePredictionService(api_key)
        st.session_state.data_loaded = False
        st.session_state.models_trained = False
        st.session_state.predictions = None

    # 데이터 로드
    if not st.session_state.data_loaded or train_button:
        with st.spinner("데이터 로드 중..."):
            data = st.session_state.service.get_interest_rate_data(
                source=data_source,
                ticker=ticker,
                start_date=start_date.strftime('%Y-%m-%d'),
                uploaded_file=uploaded_file
            )
            if data is not None:
                st.session_state.data_loaded = True

    # 데이터 탭 컨텐츠
    with data_tab:
        if st.session_state.data_loaded:
            st.subheader("금리 데이터 시각화")
            fig, ax = plt.subplots(figsize=(10, 6))
            ax.plot(st.session_state.service.data.index, st.session_state.service.data['Rate'])
            ax.set_title("금리 데이터")
            ax.set_xlabel("날짜")
            ax.set_ylabel("금리 (%)")
            ax.grid(True, alpha=0.3)
            st.pyplot(fig)

            st.subheader("통계 요약")
            st.dataframe(st.session_state.service.data.describe())

            # 이동평균 시각화
            st.subheader("이동평균 추세")
            ma_days = st.slider("이동평균 기간(일)", 7, 200, 30)

            fig2, ax2 = plt.subplots(figsize=(10, 6))
            ax2.plot(st.session_state.service.data.index, st.session_state.service.data['Rate'], label='원본 데이터')
            ax2.plot(st.session_state.service.data.index, st.session_state.service.data['Rate'].rolling(window=ma_days).mean(),
                    label=f'{ma_days}일 이동평균', color='red')
            ax2.set_title("금리 이동평균")
            ax2.set_xlabel("날짜")
            ax2.set_ylabel("금리 (%)")
            ax2.legend()
            ax2.grid(True, alpha=0.3)
            st.pyplot(fig2)

    # 모델 학습 및 예측
    if train_button and st.session_state.data_loaded:
        with st.spinner("모델 학습 중..."):
            # Prophet 모델 학습
            st.session_state.service.train_prophet_model(periods=forecast_days)

            # LSTM 모델 학습 (선택적)
            st.session_state.service.train_lstm_model(
                time_steps=time_steps,
                epochs=epochs
            )

            # 예측 수행
            st.session_state.predictions = st.session_state.service.predict_future_rates(days=forecast_days)
            st.session_state.models_trained = True

            st.success("모델 학습 및 예측 완료!")

    # 예측 결과 탭
    with forecast_tab:
        if st.session_state.models_trained and st.session_state.predictions:
            st.subheader("금리 예측 결과")

            col_curr, col_3m, col_6m, col_1y = st.columns(4)

            with col_curr:
                st.metric("현재 금리", f"{st.session_state.predictions['current_rate']}%")

            with col_3m:
                st.metric("3개월 후 예상 금리",
                         f"{st.session_state.predictions['prediction_3m']}% ± {st.session_state.predictions['confidence_3m']}",
                         delta=round(st.session_state.predictions['prediction_3m'] - st.session_state.predictions['current_rate'], 2))

            with col_6m:
                st.metric("6개월 후 예상 금리",
                         f"{st.session_state.predictions['prediction_6m']}% ± {st.session_state.predictions['confidence_6m']}",
                         delta=round(st.session_state.predictions['prediction_6m'] - st.session_state.predictions['current_rate'], 2))

            with col_1y:
                st.metric("1년 후 예상 금리",
                         f"{st.session_state.predictions['prediction_1y']}% ± {st.session_state.predictions['confidence_1y']}",
                         delta=round(st.session_state.predictions['prediction_1y'] - st.session_state.predictions['current_rate'], 2))

            st.subheader("예측 그래프")
            fig = st.session_state.service.visualize_forecast(st.session_state.predictions['forecast'])
            st.pyplot(fig)

            st.markdown(f"**금리 동향 분석**: {st.session_state.predictions['trend_description']}")

# 상담 인터페이스
with col2:
    st.header("AI 금융 상담사")

    if "messages" not in st.session_state:
        st.session_state.messages = [{"role": "assistant", "content": "안녕하세요! 금리에 관한 질문이 있으시면 언제든지 물어보세요."}]

    # 이전 메시지 표시
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # 사용자 입력 처리
    if prompt := st.chat_input("금리에 관해 질문하세요"):
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        with st.chat_message("assistant"):
            if not st.session_state.models_trained:
                response = "아직 모델이 학습되지 않았습니다. 먼저 사이드바에서 '모델 학습 시작' 버튼을 클릭해주세요."
            else:
                with st.spinner("답변 생성 중..."):
                    response = st.session_state.service.get_consultation(prompt, st.session_state.predictions)

            st.markdown(response)

        st.session_state.messages.append({"role": "assistant", "content": response})
        ''')

    # 서비스 클래스를 별도 파일로 저장
    with open('interest_rate_service.py', 'w') as f:
        f.write('''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import yfinance as yf
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
import warnings
import io
warnings.filterwarnings('ignore')

class InterestRatePredictionService:
    def __init__(self, api_key=None):
        """
        금리 예측 상담 서비스 초기화

        Parameters:
        api_key (str): OpenAI API 키
        """
        self.data = None
        self.model_prophet = None
        self.model_lstm = None
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.api_key = api_key
        self.initialize_llm()

    def initialize_llm(self):
        """LLM 및 대화 체인 초기화"""
        if self.api_key:
            template = """
            당신은 금융 전문가로서 금리에 관한 상담을 제공합니다.

            현재 상황:
            - 현재 기준금리: {current_rate}%
            - 3개월 후 예상 금리: {prediction_3m}% (신뢰구간: {confidence_3m}%)
            - 6개월 후 예상 금리: {prediction_6m}% (신뢰구간: {confidence_6m}%)
            - 1년 후 예상 금리: {prediction_1y}% (신뢰구간: {confidence_1y}%)

            최근 금리 동향: {trend_description}

            이전 대화 내용:
            {history}

            사용자: {input}
            금융 전문가:
            """

            prompt = PromptTemplate(
                input_variables=["history", "input", "current_rate", "prediction_3m",
                                "prediction_6m", "prediction_1y", "confidence_3m",
                                "confidence_6m", "confidence_1y", "trend_description"],
                template=template
            )

            self.llm = OpenAI(temperature=0.7, openai_api_key=self.api_key)
            self.memory = ConversationBufferMemory()
            self.conversation = ConversationChain(
                llm=self.llm,
                memory=self.memory,
                prompt=prompt
            )
        else:
            self.llm = None
            print("API 키가 제공되지 않았습니다. LLM 기능은 비활성화됩니다.")

    def get_interest_rate_data(self, source='fred', ticker='DGS10', start_date='2000-01-01', uploaded_file=None):
        """
        금리 데이터 수집

        Parameters:
        source (str): 데이터 소스 ('fred', 'yahoo', 또는 'upload')
        ticker (str): 데이터 티커
        start_date (str): 시작 날짜
        uploaded_file: 업로드된 CSV 파일 (선택 사항)

        Returns:
        pd.DataFrame: 금리 데이터
        """
        try:
            # 파일 업로드 옵션 처리
            if uploaded_file is not None:
                # CSV 파일 읽기
                content = uploaded_file.read().decode('utf-8')
                data = pd.read_csv(io.StringIO(content), parse_dates=True, index_col=0)
                data = data.rename(columns={data.columns[0]: 'Rate'})
            else:
                # 온라인 데이터 소스에서 가져오기
                data = yf.download(f"{ticker}=X" if source == 'yahoo' else ticker,
                                  start=start_date,
                                  end=datetime.now().strftime('%Y-%m-%d'))

                if source == 'yahoo':
                    data = data[['Close']].rename(columns={'Close': 'Rate'})
                else:
                    data = data[['Adj Close']].rename(columns={'Adj Close': 'Rate'})

            # 결측치 처리
            data = data.fillna(method='ffill')
            self.data = data
            print(f"{len(data)} 개의 데이터 포인트를 가져왔습니다.")
            return data

        except Exception as e:
            print(f"데이터 수집 중 오류 발생: {e}")
            return None

    def preprocess_data(self):
        """
        데이터 전처리 수행

        Returns:
        tuple: 전처리된 데이터 세트
        """
        if self.data is None:
            print("먼저 get_interest_rate_data 메서드를 호출하여 데이터를 가져오세요.")
            return None

        # 시계열 인덱스 확인
        self.data.index = pd.to_datetime(self.data.index)

        # Prophet 모델 준비
        prophet_data = self.data.reset_index().rename(columns={'Date': 'ds', 'Rate': 'y'})

        # LSTM 모델 준비
        dataset = self.data.values
        scaled_data = self.scaler.fit_transform(dataset)

        return prophet_data, scaled_data

    def train_prophet_model(self, periods=365):
        """
        Prophet 모델을 학습하고 예측 수행

        Parameters:
        periods (int): 예측할 미래 기간(일)

        Returns:
        pd.DataFrame: 예측 결과
        """
        prophet_data, _ = self.preprocess_data()

        # Prophet 모델 학습
        self.model_prophet = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            changepoint_prior_scale=0.05,
            seasonality_prior_scale=10.0
        )
        self.model_prophet.fit(prophet_data)

        # 미래 예측 수행
        future = self.model_prophet.make_future_dataframe(periods=periods)
        forecast = self.model_prophet.predict(future)

        print("Prophet 모델 학습 완료")
        return forecast

    def create_lstm_sequences(self, data, time_steps=60):
        """
        LSTM 모델을 위한 시퀀스 생성

        Parameters:
        data (numpy.ndarray): 스케일링된 데이터
        time_steps (int): 시퀀스 길이

        Returns:
        tuple: X, y 데이터셋
        """
        X, y = [], []
        for i in range(len(data) - time_steps):
            X.append(data[i:(i + time_steps), 0])
            y.append(data[i + time_steps, 0])
        return np.array(X), np.array(y)

    def train_lstm_model(self, time_steps=60, epochs=50, batch_size=32, validation_split=0.2):
        """
        LSTM 모델 학습

        Parameters:
        time_steps (int): 시퀀스 길이
        epochs (int): 학습 에포크 수
        batch_size (int): 배치 크기
        validation_split (float): 검증 데이터 비율

        Returns:
        tf.keras.models.Sequential: 학습된 LSTM 모델
        """
        _, scaled_data = self.preprocess_data()

        # 시퀀스 데이터 생성
        X, y = self.create_lstm_sequences(scaled_data, time_steps)

        # 훈련/테스트 분할
        train_size = int(len(X) * 0.8)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]

        # 데이터 형태 변환 (LSTM 입력용)
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

        # LSTM 모델 구성
        self.model_lstm = Sequential()
        self.model_lstm.add(LSTM(units=50, return_sequences=True, input_shape=(time_steps, 1)))
        self.model_lstm.add(Dropout(0.2))
        self.model_lstm.add(LSTM(units=50, return_sequences=False))
        self.model_lstm.add(Dropout(0.2))
        self.model_lstm.add(Dense(units=1))

        self.model_lstm.compile(optimizer='adam', loss='mean_squared_error')

        # 모델 학습
        history = self.model_lstm.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=validation_split,
            verbose=1
        )

        # 모델 평가
        y_pred = self.model_lstm.predict(X_test)
        y_pred_original = self.scaler.inverse_transform(y_pred)
        y_test_original = self.scaler.inverse_transform(y_test.reshape(-1, 1))

        mse = mean_squared_error(y_test_original, y_pred_original)
        mae = mean_absolute_error(y_test_original, y_pred_original)

        print(f"LSTM 모델 학습 완료")
        print(f"테스트 세트 MSE: {mse:.4f}, MAE: {mae:.4f}")

        return self.model_lstm, history

    def predict_future_rates(self, days=365):
        """
        Prophet 및 LSTM 모델을 사용한 금리 예측

        Parameters:
        days (int): 예측할 미래 기간(일)

        Returns:
        dict: 예측 결과
        """
        if self.model_prophet is None:
            print("먼저 Prophet 모델을 학습하세요.")
            return None

        # Prophet 예측 결과
        forecast = self.model_prophet.predict(self.model_prophet.make_future_dataframe(periods=days))

        # 현재 금리, 3개월, 6개월, 1년 후 예측
        current_date = datetime.now().strftime('%Y-%m-%d')
        current_rate = forecast[forecast['ds'] == current_date]['yhat'].values[0] if current_date in forecast['ds'].dt.strftime('%Y-%m-%d').values else forecast.iloc[-len(forecast) + len(self.data)]['yhat']

        prediction_3m = forecast[forecast['ds'] == (datetime.now() + timedelta(days=90)).strftime('%Y-%m-%d')]['yhat'].values[0]
        prediction_6m = forecast[forecast['ds'] == (datetime.now() + timedelta(days=180)).strftime('%Y-%m-%d')]['yhat'].values[0]
        prediction_1y = forecast[forecast['ds'] == (datetime.now() + timedelta(days=365)).strftime('%Y-%m-%d')]['yhat'].values[0]

        # 신뢰 구간 계산 (Prophet)
        confidence_3m = (forecast[forecast['ds'] == (datetime.now() + timedelta(days=90)).strftime('%Y-%m-%d')]['yhat_upper'].values[0] -
                         forecast[forecast['ds'] == (datetime.now() + timedelta(days=90)).strftime('%Y-%m-%d')]['yhat_lower'].values[0]) / 2
        confidence_6m = (forecast[forecast['ds'] == (datetime.now() + timedelta(days=180)).strftime('%Y-%m-%d')]['yhat_upper'].values[0] -
                         forecast[forecast['ds'] == (datetime.now() + timedelta(days=180)).strftime('%Y-%m-%d')]['yhat_lower'].values[0]) / 2
        confidence_1y = (forecast[forecast['ds'] == (datetime.now() + timedelta(days=365)).strftime('%Y-%m-%d')]['yhat_upper'].values[0] -
                         forecast[forecast['ds'] == (datetime.now() + timedelta(days=365)).strftime('%Y-%m-%d')]['yhat_lower'].values[0]) / 2

        # 금리 동향 설명
        last_3m_data = self.data.iloc[-90:] if len(self.data) >= 90 else self.data
        trend = "상승" if last_3m_data['Rate'].iloc[-1] > last_3m_data['Rate'].iloc[0] else "하락"
        avg_change = (last_3m_data['Rate'].iloc[-1] - last_3m_data['Rate'].iloc[0]) / len(last_3m_data) * 100

        trend_description = f"최근 3개월 동안 금리는 {trend} 추세를 보이고 있으며, 평균 일일 변화율은 {avg_change:.4f}%입니다."

        predictions = {
            'current_rate': round(current_rate, 2),
            'prediction_3m': round(prediction_3m, 2),
            'prediction_6m': round(prediction_6m, 2),
            'prediction_1y': round(prediction_1y, 2),
            'confidence_3m': round(confidence_3m, 2),
            'confidence_6m': round(confidence_6m, 2),
            'confidence_1y': round(confidence_1y, 2),
            'trend_description': trend_description,
            'forecast': forecast
        }

        return predictions

    def visualize_forecast(self, forecast, days_to_show=365, save_path=None):
        """
        예측 결과 시각화

        Parameters:
        forecast (pd.DataFrame): Prophet 예측 결과
        days_to_show (int): 표시할 기간(일)
        save_path (str): 이미지 저장 경로

        Returns:
        matplotlib.figure.Figure: 그래프 피겨
        """
        plt.figure(figsize=(14, 8))

        # 실제 데이터 플롯
        plt.plot(self.data.index[-days_to_show:],
                self.data['Rate'].values[-days_to_show:],
                label='실제 금리', color='blue')

        # 예측 데이터 플롯 (현재부터)
        future_dates = pd.date_range(start=datetime.now(), periods=365)
        future_idx = [i for i, date in enumerate(forecast['ds']) if date >= datetime.now()]

        plt.plot(forecast['ds'].iloc[future_idx],
                forecast['yhat'].iloc[future_idx],
                label='예측 금리', color='red', linestyle='--')

        # 신뢰 구간 표시
        plt.fill_between(forecast['ds'].iloc[future_idx],
                         forecast['yhat_lower'].iloc[future_idx],
                         forecast['yhat_upper'].iloc[future_idx],
                         color='red', alpha=0.2, label='95% 신뢰 구간')

        plt.title('금리 예측 그래프', fontsize=16)
        plt.xlabel('날짜', fontsize=12)
        plt.ylabel('금리 (%)', fontsize=12)
        plt.legend()
        plt.grid(True, alpha=0.3)

        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')

        return plt.gcf()

    def get_consultation(self, user_query, predictions=None):
        """
        사용자 쿼리에 기반한 상담 제공

        Parameters:
        user_query (str): 사용자 질문
        predictions (dict): 예측 결과

        Returns:
        str: LLM 기반 응답
        """
        if self.llm is None:
            return "API 키가 설정되지 않아 LLM 상담 기능을 사용할 수 없습니다."

        if predictions is None:
            predictions = self.predict_future_rates()

        if predictions is None:
            return "먼저 모델을 학습하고 예측을 수행해야 합니다."

        try:
            response = self.conversation.predict(
                input=user_query,
                current_rate=predictions['current_rate'],
                prediction_3m=predictions['prediction_3m'],
                prediction_6m=predictions['prediction_6m'],
                prediction_1y=predictions['prediction_1y'],
                confidence_3m=predictions['confidence_3m'],
                confidence_6m=predictions['confidence_6m'],
                confidence_1y=predictions['confidence_1y'],
                trend_description=predictions['trend_description']
            )
            return response
        except Exception as e:
            return f"상담 제공 중 오류가 발생했습니다: {e}"
''')

    # StreamlitTunnel 시작 - ngrok을 통해 외부 접속 허용
    !streamlit run app.py &> /dev/null &

    # 잠시 대기하여 Streamlit이 포트를 열 수 있도록 함
    import time
    time.sleep(5)

    # ngrok으로 포트 연결 (올바른 방식으로 수정)
    from pyngrok import ngrok

    # 이전 터널 모두 종료
    ngrok.kill()

    # 명시적으로 포트 지정하여 HTTP 터널 생성
    public_url = ngrok.connect(addr=8501, proto="http")
    print(f"Streamlit 애플리케이션이 다음 URL에서 실행 중입니다: {public_url}")

In [ ]:
# 'download_model' 버튼 추가 - 학습된 모델 다운로드 기능
def download_trained_model(service):
    if service and service.model_prophet and service.model_lstm:
        # Prophet 모델 저장
        with open('prophet_model.json', 'w') as f:
            f.write(service.model_prophet.to_json())

        # LSTM 모델 저장
        service.model_lstm.save('lstm_model')

        # 모델 파일 압축
        !zip -r models.zip prophet_model.json lstm_model

        # 압축 파일 다운로드
        files.download('models.zip')
    else:
        print("먼저 모델을 학습해야 합니다.")

In [ ]:
from pyngrok import ngrok, conf
ngrok.set_auth_token("1ou9hCwZym4yBH78OO3gSuUjuMC_6hG8KvaSsRuCPVjsmjDAC")

In [ ]:
# 메인 함수
def main():
    print("Google Colab에서 금리 예측 상담 서비스 실행 방법:")
    print("1. Streamlit 웹 앱으로 실행 (ngrok 사용)")
    print("2. Colab 노트북에서 직접 실행")

    choice = int(input("선택하세요 (1 또는 2): "))

    if choice == 1:
        run_streamlit_on_colab()
    elif choice == 2:
        run_directly_in_colab()
    else:
        print("잘못된 선택입니다. 1 또는 2를 입력하세요.")

if __name__ == "__main__":
    main()

Google Colab에서 금리 예측 상담 서비스 실행 방법:
1. Streamlit 웹 앱으로 실행 (ngrok 사용)
2. Colab 노트북에서 직접 실행
선택하세요 (1 또는 2): 1
Streamlit 애플리케이션이 다음 URL에서 실행 중입니다: NgrokTunnel: "https://3232-34-106-31-217.ngrok-free.app" -> "http://localhost:8501"
